In [1]:
# import required packages
# from fastai2.tabular.all import *
import xgboost as xgb
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from helpers import *

pd.options.display.max_columns = None

In [2]:
# path to project directory
path = Path('./')

In [3]:
# read in training dataset
train_df = pd.read_csv(path/'data/train_v7.csv', index_col=0, dtype={'season':str,
                                                                     'squad':str,
                                                                     'comp':str})

In [4]:
train_df.shape

(108238, 58)

In [5]:
# features for model
season = '2021'
cat_vars = ['season', 'position', 'was_home']
cont_vars = ['gw', 'minutes']
dep_var = ['total_points']

# find the latest gameweek
last_gw = train_df['gw'][train_df['season'] == season].max()

if np.isnan(last_gw): 
    next_gw = 1 
else: 
    next_gw = last_gw + 1
    
next_gw

29

In [6]:
# read in remaining_season.csv and sort
remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
                                  index_col=0).sort_values(by=['gw', 'team'])

# update fields needed for processing
remaining_season_df['season'] = season
remaining_season_df['total_points'] = 0

# need to add an ordered date-like field  
remaining_season_df['kickoff_time'] = [str(9000 + x) + '-99-99' for x in remaining_season_df['match_no']]

# remove match_no, don't need it anymore
remaining_season_df = remaining_season_df.drop('match_no', axis=1)

# # append remaining season to end of training set
train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [7]:
# # read in remaining_season.csv and sort
# remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
#                                   index_col=0).sort_values(by=['gw', 'team'])

# remaining_season_df['season'] = season
# remaining_season_df['total_points'] = 0
# remaining_season_df['kickoff_time'] = '9999-99-99'

# # append remaining season to end of training set
# train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [8]:
# add player lag features
lag_train_df, team_lag_vars = team_lag_features(train_df, ['total_points'], ['all', 3, 5, 10, 20])
lag_train_df, player_lag_vars = player_lag_features(lag_train_df, ['total_points'], ['all', 1, 2, 3, 5, 10, 20])

# add team conceded fields for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars += [x.replace('team', 'team_conceded') for x in team_lag_vars if pattern.match(x)]

# but remove scored for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars = [x for x in team_lag_vars if not pattern.match(x)]

In [9]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [10]:
# we want to set gw and season as ordered categorical variables
# need lists with ordered categories
ordered_gws = list(range(1,39))
ordered_seasons = ['1617', '1718', '1819', '1920', '2021']

# set as categories with correct order 
lag_train_df['gw'] = lag_train_df['gw'].astype('category')
lag_train_df['season'] = lag_train_df['season'].astype('category')

lag_train_df['gw'].cat.set_categories(ordered_gws, ordered=True, inplace=True)
lag_train_df['season'].cat.set_categories(ordered_seasons, ordered=True, inplace=True)

In [11]:
lag_train_df[lag_train_df['player'] == 'Bernd Leno'].tail(25)

player  gw  position  minutes     team             opponent_team  \
97918   Bernd Leno  14         1     90.0  Arsenal                   Everton   
98530   Bernd Leno  15         1     90.0  Arsenal                   Chelsea   
99123   Bernd Leno  16         1     90.0  Arsenal  Brighton and Hove Albion   
99622   Bernd Leno  17         1     90.0  Arsenal      West Bromwich Albion   
100175  Bernd Leno  18         1     90.0  Arsenal            Crystal Palace   
100634  Bernd Leno  19         1     90.0  Arsenal          Newcastle United   
101553  Bernd Leno  20         1     90.0  Arsenal               Southampton   
102198  Bernd Leno  21         1     90.0  Arsenal         Manchester United   
102849  Bernd Leno  22         1     71.0  Arsenal   Wolverhampton Wanderers   
103512  Bernd Leno  23         1      0.0  Arsenal               Aston Villa   
104205  Bernd Leno  24         1     90.0  Arsenal                     Leeds   
104987  Bernd Leno  25         1     90.0  Arsenal           Manchester City   
105779  Bernd Leno  26         1     90.0  Arsenal            Leicester City   
106886  Bernd Leno  27         1     90.0  Arsenal                   Burnley   
107633  Bernd Leno  28         1     90.0  Arsenal         Tottenham Hotspur   
108245  Bernd Leno  29         1     90.0  Arsenal           West Ham United   
108522  Bernd Leno  30         1     90.0  Arsenal                 Liverpool   
109208  Bernd Leno  31         1     90.0  Arsenal          Sheffield United   
109894  Bernd Leno  32         1     90.0  Arsenal                    Fulham   
110580  Bernd Leno  33         1     90.0  Arsenal                   Everton   
111128  Bernd Leno  34         1     90.0  Arsenal          Newcastle United   
111953  Bernd Leno  35         1     90.0  Arsenal      West Bromwich Albion   
112639  Bernd Leno  36         1     90.0  Arsenal                   Chelsea   
113325  Bernd Leno  37         1     90.0  Arsenal            Crystal Palace   
114011  Bernd Leno  38         1     90.0  Arsenal  Brighton and Hove Albion   

        relative_market_value_team  relative_market_value_opponent_team  \
97918                     1.317380                             1.145049   
98530                     1.317380                             1.859440   
99123                     1.316625                             0.541862   
99622                     1.261632                             0.258537   
100175                    1.264284                             0.475724   
100634                    1.272515                             0.604033   
101553                    1.343890                             0.538945   
102198                    1.353529                             1.679166   
102849                    1.253765                             0.969065   
103512                    1.253765                             0.749770   
104205                    1.252915                             0.439832   
104987                    1.252915                             2.397273   
105779                    1.252915                             1.059550   
106886                    1.252936                             0.342974   
107633                    1.279916                             1.586900   
108245                    1.279916                             0.652079   
108522                    1.279916                             2.354244   
109208                    1.279916                             0.356167   
109894                    1.279916                             0.520031   
110580                    1.279916                             1.078057   
111128                    1.279916                             0.589027   
111953                    1.279916                             0.288453   
112639                    1.279916                             1.817663   
113325                    1.279916                             0.473879   
114011                    1.279916                      

In [12]:
# create dataset with latest lag numbers for all future gameweeks
valid_season = '2021'
valid_gw = next_gw
valid_len = 38 - next_gw + 1

train_predict_df, train_idx, valid_idx = create_lag_train(lag_train_df, 
                                                      cat_vars, cont_vars, 
                                                      player_lag_vars, team_lag_vars, dep_var,
                                                      valid_season, valid_gw, valid_len)

In [13]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [14]:
# take a look at the dataframe
train_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114690 entries, 0 to 114689
Data columns (total 33 columns):
 #   Column                                           Non-Null Count   Dtype   
---  ------                                           --------------   -----   
 0   gw                                               114690 non-null  category
 1   minutes                                          114690 non-null  float64 
 2   minutes_last_1                                   114690 non-null  float64 
 3   minutes_last_10                                  114690 non-null  float64 
 4   minutes_last_2                                   114690 non-null  float64 
 5   minutes_last_20                                  114690 non-null  float64 
 6   minutes_last_3                                   114690 non-null  float64 
 7   minutes_last_5                                   114690 non-null  float64 
 8   minutes_last_all                                 114690 non-null  float64 
 9   oppo

In [15]:
# split out dependent variable
X, y = train_predict_df[cat_vars + cont_vars + player_lag_vars + team_lag_vars].copy(), train_predict_df[dep_var].copy()

In [16]:
# since position is categorical, it should be a string
X['position'] = X['position'].apply(str)

# need to transform season
enc = LabelEncoder()
X['season'] = enc.fit_transform(X['season'])
X_dict = X.to_dict("records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False, separator='_')

# Apply dv on df: df_encoded
X_encoded = dv.fit_transform(X_dict)

X_df = pd.DataFrame(X_encoded, columns=dv.feature_names_)

In [17]:
# split out training and prediction sets
X_train = X_df.iloc[train_idx]
y_train = y.iloc[train_idx]
X_test = X_df.iloc[valid_idx]
y_test = y.iloc[valid_idx]

In [18]:
# instatiate and train XGB Regressor
model_params = {"gamma": 0.42,
                "learning_rate": 0.047, 
                "max_depth": 4,
                "n_estimators": 171,
                "subsample": 0.6}

xg_reg = xgb.XGBRegressor(objective="reg:squarederror",
                          gamma=model_params['gamma'], 
                          learning_rate=model_params['learning_rate'], 
                          max_depth=model_params['max_depth'],
                          n_estimators=model_params['n_estimators'],
                          subsample=model_params['subsample'])

xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)

In [19]:
preds

array([ 1.4070660e-02, -2.8342605e-03,  3.2161462e+00, ...,
       -7.5727701e-03,  2.3015594e+00, -7.5727701e-03], dtype=float32)

In [20]:
remaining_season_df['predicted_points'] = preds

In [21]:
remaining_season_df.head(50)

gw         team      opponent_team                            player  \
3332  29      Arsenal    West Ham United                        Mesut Özil   
3333  29      Arsenal    West Ham United         Sokratis Papastathopoulos   
3334  29      Arsenal    West Ham United        David Luiz Moreira Marinho   
3335  29      Arsenal    West Ham United         Pierre-Emerick Aubameyang   
3336  29      Arsenal    West Ham United                     Cédric Soares   
3337  29      Arsenal    West Ham United               Alexandre Lacazette   
3338  29      Arsenal    West Ham United                  Shkodran Mustafi   
3339  29      Arsenal    West Ham United                        Bernd Leno   
3340  29      Arsenal    West Ham United                      Granit Xhaka   
3341  29      Arsenal    West Ham United                        Pablo Marí   
3342  29      Arsenal    West Ham United                   Héctor Bellerín   
3343  29      Arsenal    West Ham United                    Calum Chambers   
3344  29      Arsenal    West Ham United                    Sead Kolasinac   
3345  29      Arsenal    West Ham United                       Rob Holding   
3346  29      Arsenal    West Ham United                    Kieran Tierney   
3347  29      Arsenal    West Ham United                      Nicolas Pépé   
3348  29      Arsenal    West Ham United                    Lucas Torreira   
3349  29      Arsenal    West Ham United                      Reiss Nelson   
3350  29      Arsenal    West Ham United                    Edward Nketiah   
3351  29      Arsenal    West Ham United                  Emile Smith Rowe   
3352  29      Arsenal    West Ham United                       Bukayo Saka   
3353  29      Arsenal    West Ham United                  Matteo Guendouzi   
3354  29      Arsenal    West Ham United  Gabriel Teodoro Martinelli Silva   
3355  29      Arsenal    West Ham United                    William Saliba   
3356  29      Arsenal    West Ham United                       Mathew Ryan   
3357  29      Arsenal    West Ham United           Willian Borges Da Silva   
3358  29      Arsenal    West Ham United                 Gabriel Magalhães   
3359  29      Arsenal    West Ham United         Daniel Ceballos Fernández   
3360  29      Arsenal    West Ham United     Mohamed Naser El Sayed Elneny   
3361  29      Arsenal    West Ham United                        Matt Macey   
3362  29      Arsenal    West Ham United              Rúnar Alex Rúnarsson   
3363  29      Arsenal    West Ham United                     Thomas Partey   
3364  29      Arsenal    West Ham United                   Martin Ødegaard   
0     29  Aston Villa  Tottenham Hotspur                 Emiliano Martínez   
1     29  Aston Villa  Tottenham Hotspur                        Tom Heaton   
2     29  Aston Villa  Tottenham Hotspur                 Ahmed El Mohamady   
3     29  Aston Villa  Tottenham Hotspur                    Henri Lansbury   
4     29  Aston Villa  Tottenham Hotspur                       Neil Taylor   
5     29  Aston Villa  Tottenham Hotspur                         Jed Steer   
6     29  Aston Villa  Tottenham Hotspur                   Conor Hourihane   
7     29  Aston Villa  Tottenham Hotspur    José Ignacio Peleteiro Romallo   
8     29  Aston Villa  Tottenham Hotspur                      Ørjan Nyland   
9     29  Aston Villa  Tottenham Hotspur                      Bjorn Engels   
10    29  Aston Villa  Tottenham Hotspur                     Jack Grealish   
11    29  Aston Villa  Tottenham Hotspur                       John McGinn   
12    29  Aston Villa  Tottenham Hotspur                     Kortney Hause   
13    29  Aston Villa  Tottenham Hotspur      Mahmoud Ahmed Ibrahim Hassan   
14    29  Aston Villa  Tottenham Hotspur                      Tyrone Mings   
15    29  Aston Villa  Tottenham Hotspur                      Matt Targett   
16    29  Aston Villa  Tottenham Hotspur                        James Bree   

      position  price  play

In [22]:
remaining_season_df[remaining_season_df['gw'] == next_gw].sort_values(by='predicted_points', ascending=False).head(50)

gw                      team             opponent_team  \
3237  29         Tottenham Hotspur               Aston Villa   
48    29  Brighton and Hove Albion          Newcastle United   
3326  29                     Leeds                    Fulham   
3318  29                     Leeds                    Fulham   
3305  29                     Leeds                    Fulham   
49    29  Brighton and Hove Albion          Newcastle United   
37    29  Brighton and Hove Albion          Newcastle United   
120   29           West Ham United                   Arsenal   
104   29                    Fulham                     Leeds   
105   29                    Fulham                     Leeds   
114   29           West Ham United                   Arsenal   
3232  29         Tottenham Hotspur               Aston Villa   
3259  29         Tottenham Hotspur               Aston Villa   
110   29           West Ham United                   Arsenal   
42    29  Brighton and Hove Albion          Newcastle United   
0     29               Aston Villa         Tottenham Hotspur   
65    29  Brighton and Hove Albion          Newcastle United   
101   29                    Fulham                     Leeds   
28    29               Aston Villa         Tottenham Hotspur   
47    29  Brighton and Hove Albion          Newcastle United   
3339  29                   Arsenal           West Ham United   
3302  29                     Leeds                    Fulham   
107   29                    Fulham                     Leeds   
3241  29         Tottenham Hotspur               Aston Villa   
53    29  Brighton and Hove Albion          Newcastle United   
3264  29          Newcastle United  Brighton and Hove Albion   
106   29                    Fulham                     Leeds   
3364  29                   Arsenal           West Ham United   
3309  29                     Leeds                    Fulham   
75    29                    Fulham                     Leeds   
3346  29                   Arsenal           West Ham United   
3363  29                   Arsenal           West Ham United   
131   29           West Ham United                   Arsenal   
3307  29                     Leeds                    Fulham   
109   29                    Fulham                     Leeds   
55    29  Brighton and Hove Albion          Newcastle United   
3334  29                   Arsenal           West Ham United   
3253  29         Tottenham Hotspur               Aston Villa   
135   29           West Ham United                   Arsenal   
3258  29         Tottenham Hotspur               Aston Villa   
3266  29          Newcastle United  Brighton and Hove Albion   
98    29                    Fulham                     Leeds   
3273  29          Newcastle United  Brighton and Hove Albion   
3246  29         Tottenham Hotspur               Aston Villa   
3340  29                   Arsenal           West Ham United   
118   29           West Ham United                   Arsenal   
3325  29                     Leeds                    Fulham   
3278  29          Newcastle United  Brighton and Hove Albion   
3311  29                     Leeds                    Fulham   
2     29               Aston Villa         Tottenham Hotspur   

                              player  position  price  play_proba  \
3237                      Harry Kane         4   11.5        1.00   
48                       Neal Maupay         4    6.1        1.00   
3326            Raphael Dias Belloli         3    5.6        1.00   
3318                   Illan Meslier         1    4.6        1.00   
3305                   Stuart Dallas         2    5.1        1.00   
49                  Leandro Trossard         3    5.8        1.00   
37                       Pascal Groß         3    5.8        1.00   
120                  Michail Antonio         4    6.7        1.00   
104                  Ademola Lookman         3    5.1        1.00   
105                         Ola Aina         2    4

In [23]:
# rename previous week's predictions file
# generate previous week's filename
last_gw = next_gw - 1
filename = 'history/2020-21/predictions_gw' + str(last_gw) + '.csv'
# windows
! move predictions.csv $filename
# linux
# ! mv predictions.csv $filename

        1 file(s) moved.


In [24]:
# write to predictions.csv
## RENAME PREVIOUS WEEK FIRST
predictions = remaining_season_df[['gw', 'player', 'team', 'opponent_team', 
                                   'position', 'price', 'play_proba', 'short_name',
                                   'was_home', 'predicted_points', 'minutes']]
predictions.to_csv('predictions.csv')